# Posterior HMM Decoding Assignment

This is not due until after the Viterbi decoding lab. Please do not start this lab until you've completed that one.

## Part 1: Implement posterior decoding

The file input-output and the input-output of the top level decoding function, `posterior_decode`, are the same as for the Viterbi decoding lab. The representations of HMMs is also the same. Instead of the single function `_build_matrix` from Viterbi, you will have two: `_build_forward_matrix` and `_build_backward_matrix`. You will then combine them inside `_posterior_probabilities`. `posterior_decode` calls `_posterior_probabilities` and uses the result to find the posterior path and output the corresponding state names.

Because the posterior decoding only cares about the relative probabilities of the states at each time, you can multiply all the forward probabilities for a given observation or all the backward probabilities or both by arbitrary positive constants without changing the path. Thus, you should normalize each column of the forward and backward matrices as you build them. You should also normalize the product of the two, so that at the end of the calculation you have the posterior probabilities of the states for each observation.

I suggest copying your `_build_matrix` from the Viterbi lab as the basis for `_build_forward_matrix`. Aside from renaming variables to be more appropriate, there is only one, very small, substantive change to the way entries are calculated.

You can then copy `_build_forward_matrix` as the basis for `_build_backward_matrix`. Here some substantive changes are required, but the overall structure is the same. Differences include the initialization of the row for the last observation, the fact that you count backwards from the end of the matrix, and the actual calculation of the entries at each time point.

## Part 2: Difference between Viterbi and Posterior

**Question 1:** Does posterior decoding of "Data/mixed2.fa" with HMM "Data/humanMalaria.hmm" give a different result than Viterbi decoding? Write and evaluate your code for determining the answer in the cells below. Before you do that, copy assignment.py from your correct Viterbi assignment into this directory with the name viterbi_assignment.py (with an underscore not a hyphen). The provided code below will then load it. You can download and upload files from codespaces using the Explorer pane at the left. If you don't have a correct viterbi implementation, you should work on getting that right first.

In [ ]:
# 5 points
import os
import numpy as np
from cse587Autils.HMMObjects.HMM import HMM, calculate_accuracy
from assignment import posterior_decode, _posterior_probabilities

# Import viterbi_decode from the local viterbi_assignment.py, which should
# be in the same directory as this notebook. If it isn't, put it there first.

from viterbi_assignment import viterbi_decode

# Get the path to the Data directory
DATA_DIR = os.path.join(os.getcwd(), "Data")

# Load the HMM and sequence
hmm = HMM.read_hmm_file(os.path.join(DATA_DIR, "humanMalaria.hmm"))
sequences = HMM.read_fasta(os.path.join(DATA_DIR, "mixed2.fa"))
observation_seq = sequences[0]

# YOUR CODE HERE FOR POSTERIOR DECODE, VITERBI DECODE, AND COMPARISON
# <snip>

posterior_result = posterior_decode(observation_seq, hmm)

viterbi_result = viterbi_decode(observation_seq, hmm)
    
differences = sum(1 for i in range(len(posterior_result)) if posterior_result[i] != viterbi_result[i])
print(f"Number of positions where Viterbi and Posterior differ: {differences}")
print(f"Percentage different: {100 * differences / len(observation_seq):.2f}%")

# Calculate accuracy for posterior
key_seq = HMM.read_fasta(os.path.join(DATA_DIR, "mixed2key.fa"))[0]
accuracy = calculate_accuracy(posterior_result, key_seq)
print(f"\nPosterior accuracy: {accuracy} out of {len(observation_seq)} ({100*accuracy/len(observation_seq):.2f}%)")
# </snip>

**Question 2:** Use `_posterior_probabilities` to calculate the state probabilities for each state in the decoding of "Data/mixed2.fa" with HMM "Data/humanMalaria.hmm". How far away from 0.5 does it get (Max absolute difference)? How close to 0.5 does it get (Min absolute difference)? Would you say that the algorithm is very confident of its state calls overall, or not very confident?

In [ ]:
# 5 points
# YOUR CODE HERE
# <snip>
# Calculate posterior probabilities
posterior_probs = _posterior_probabilities(observation_seq, hmm);

# For a 2-state HMM, if one state has probability p, the other has 1-p
# Distance from 0.5 tells us about confidence
# We'll look at the first state's probabilities
state_probs = posterior_probs[:, 0]

# Calculate distances from 0.5
distances = np.abs(state_probs - 0.5)

max_distance = np.max(distances)
min_distance = np.min(distances)

# </snip>
print(f"Max absolute distance from 0.5: {max_distance:.6f}")
print(f"Min absolute distance from 0.5: {min_distance:.6f}")
print(f"\nMean distance from 0.5: {np.mean(distances):.6f}")
print(f"Median distance from 0.5: {np.median(distances):.6f}")


**Question 3:** In the Viterbi decoding lab, you created a file "tweakedHMM.hmm" 
to try to get better accuracy than humanMalaria.hmm. Repeat questions 1 and 2 
above for that HMM. Do Viterbi and posterior decoding give the same result? Is 
the posterior decoding more confident or less than it was with original HMM,
humanMalaria.hmm? Before writing code, copy the "tweakedHMM.hmm" file into 
the Data subdirectory of this assignment.

In [ ]:
# 6 points
tweaked_hmm_path = os.path.join(DATA_DIR, "tweakedHMM.hmm")
tweaked_hmm = HMM.read_hmm_file(tweaked_hmm_path)

 # YOUR CODE HERE
 # <snip>   
posterior_tweaked = posterior_decode(observation_seq, tweaked_hmm)
viterbi_tweaked = viterbi_decode(observation_seq, tweaked_hmm)
differences = sum(1 for i in range(len(posterior_tweaked)) if posterior_tweaked[i] != viterbi_tweaked[i])
print(f"Differences between Viterbi and Posterior: {differences} ({100*differences/len(observation_seq):.2f}%)")

# Calculate accuracy
accuracy_tweaked = calculate_accuracy(posterior_tweaked, key_seq)
print(f"Posterior accuracy with tweaked HMM: {accuracy_tweaked} ({100*accuracy_tweaked/len(observation_seq):.2f}%)")
    
# Analyze confidence
posterior_probs_tweaked = _posterior_probabilities(observation_seq, tweaked_hmm);
distances_tweaked = np.abs(posterior_probs_tweaked[:, 0] - 0.5)

# </snip>
print(f"\nMax distance from 0.5: {np.max(distances_tweaked):.6f}")
print(f"Min distance from 0.5: {np.min(distances_tweaked):.6f}")
print(f"Mean distance from 0.5: {np.mean(distances_tweaked):.6f}")
print(f"\nCompared to humanMalaria.hmm mean distance of {np.mean(distances):.6f}")

**Question 4:** Make a file called **testHMM7.hmm** in the Data directory. 
Design it so that posterior and Viterbi decoding get different results when you 
evaluate the following calls. Below that, write a sentence or two explaining how 
you approached the problem.

**Hints:**
- you may need more than 2 or 3 states, but it can definitely be done with 4.
- In designing your HMM, think about the difference between what Viterbi and posterior decoding are trying to do.

In [ ]:
# Load and validate testHMM7.hmm
hmm7_path = os.path.join(DATA_DIR, "testHMM7.hmm")
hmm7 = HMM.read_hmm_file(hmm7_path)
print("HMM validity check:", hmm7.check_validity())

In [ ]:
# Test with Viterbi
test_seq = [0, 0, 0, 0, 0, 0, 0, 0, 0]  # Converted from Mathematica {1,1,1,1,1,1,1,1,1}
viterbi_result_7 = viterbi_decode(test_seq, hmm7)
print("Viterbi result:", viterbi_result_7)

In [ ]:
# Test with Posterior
posterior_result_7 = posterior_decode(test_seq, hmm7)
print("Posterior result:", posterior_result_7)

**Explanation of approach for testHMM7:**

*Write your explanation here about how you designed the HMM to produce different results between Viterbi and Posterior decoding. Consider:*
- *What is the key difference between what Viterbi optimizes (best single path) vs. Posterior (marginal probability at each position)?*
- *How can transition probabilities create a situation where the globally best path differs from the locally best state assignments?*